## Innomatics Data Science with GenAI Internship Test

In [1]:
import pandas as pd

# Load the uploaded files
photos_data_path = 'property_photos.tsv'
interactions_data_path = 'property_interactions.csv'

# Reading the files into DataFrames
photos_data = pd.read_csv(photos_data_path, sep='\t')  # Assuming it's a TSV file
interactions_data = pd.read_csv(interactions_data_path)

# Displaying the first few rows of each dataset to understand their structure
photos_data_head = photos_data.head()
interactions_data_head = interactions_data.head()

photos_data_info = photos_data.info()
interactions_data_info = interactions_data.info()

photos_data_head, interactions_data_head, photos_data_info, interactions_data_info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28888 entries, 0 to 28887
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  28888 non-null  object
 1   photo_urls   17866 non-null  object
dtypes: object(2)
memory usage: 451.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170611 entries, 0 to 170610
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   property_id   170611 non-null  object
 1   request_date  170611 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


(                        property_id  \
 0  ff808081469fd6e20146a5af948000ea   
 1  ff8080814702d3d10147068359d200cd   
 2  ff808081470c645401470fb03f5800a6   
 3  ff808081470c6454014715eaa5960281   
 4  ff808081474aa867014771a0298f0aa6   
 
                                           photo_urls  
 0  [{\title\":\"Balcony\",\"name\":\"IMG_20131006...  
 1                                                NaN  
 2  [{\title\":\"Bedroom\",\"name\":\"ff808081470c...  
 3  [{\title\":\"Bedroom\",\"name\":\"Screenshot_7...  
 4  [{\title\":\"Other\",\"name\":\"ff808081474aa8...  ,
                         property_id         request_date
 0  ff808081469fd6e20146a5af948000ea  2017-03-10 17:42:34
 1  ff808081469fd6e20146a5af948000ea  2017-03-09 15:51:17
 2  ff808081469fd6e20146a5af948000ea  2017-03-10 17:30:22
 3  ff808081469fd6e20146a5af948000ea  2017-03-11 17:48:46
 4  ff8080814702d3d10147068359d200cd  2017-03-30 19:59:15,
 None,
 None)

In [2]:
import ast

# Cleaning and extracting photo counts from the photo_urls column
def extract_photo_count(photo_urls):
    if pd.isna(photo_urls):  # Handle missing values
        return 0
    try:
        photos = ast.literal_eval(photo_urls)
        return len(photos)
    except (ValueError, SyntaxError):  # Handle malformed data
        return 0

photos_data['photo_count'] = photos_data['photo_urls'].apply(extract_photo_count)

# Aggregating total interactions for each property
interactions_data['request_date'] = pd.to_datetime(interactions_data['request_date'])
total_interactions = interactions_data.groupby('property_id').size().reset_index(name='total_interactions')

# Merging photo data with interactions data
merged_data = photos_data[['property_id', 'photo_count']].merge(
    total_interactions, on='property_id', how='outer'
)

# Checking the result of the merging process
merged_data_info = merged_data.info()
merged_data_head = merged_data.head()

merged_data_info, merged_data_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28888 entries, 0 to 28887
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   property_id         28888 non-null  object 
 1   photo_count         28888 non-null  int64  
 2   total_interactions  22779 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 677.2+ KB


(None,
                         property_id  photo_count  total_interactions
 0  ff808081469fd6e20146a5af948000ea            0                 4.0
 1  ff8080814702d3d10147068359d200cd            0                12.0
 2  ff808081470c645401470fb03f5800a6            0                27.0
 3  ff808081470c6454014715eaa5960281            0                13.0
 4  ff808081474aa867014771a0298f0aa6            0                68.0)

In [3]:
# Filling missing values in total_interactions with 0 for analysis
merged_data['total_interactions'].fillna(0, inplace=True)

# Calculate summary statistics needed for the questions
hsr_properties_count = merged_data[merged_data['property_id'].str.contains('HSR', na=False)].shape[0]
total_properties = merged_data.shape[0]
hsr_percentage = round((hsr_properties_count / total_properties) * 100, 2)

# Count unique locations from property_ids (assuming the location is encoded in property_id)
unique_locations_count = merged_data['property_id'].str.extract(r'(\D+)').nunique()[0]

# Summarize results for next tasks
hsr_percentage, unique_locations_count

C:\Users\prasad jadhav\AppData\Local\Temp\ipykernel_1988\3767893106.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['total_interactions'].fillna(0, inplace=True)


(0.0, 1)

In [4]:
# Revisiting the data: Checking property_id patterns for extracting location information
property_id_sample = merged_data['property_id'].dropna().head(10)

# Display sample property IDs for further analysis
property_id_sample

0    ff808081469fd6e20146a5af948000ea
1    ff8080814702d3d10147068359d200cd
2    ff808081470c645401470fb03f5800a6
3    ff808081470c6454014715eaa5960281
4    ff808081474aa867014771a0298f0aa6
5    ff8080814780f30801478d14132f03f4
6    ff8080814780f308014797c79000082e
7    ff80808147a95f0a0147ab0d88fa0080
8    ff80808147ac6af10147afc279480099
9    ff80808147cbdc2a0147cf8577880153
Name: property_id, dtype: object

### Insights from the uploaded datasets:

#### Property Photos Data:
- #### Columns:
- property_id: Unique identifier for each property.
- photo_urls: JSON-like strings containing photo details for each property.
- #### Shape: 28,888 rows and 2 columns.

- #### Non-Null Count:
- property_id: Fully populated.
- photo_urls: Contains missing values (17,866 non-null entries).
- Property Interactions Data:

- #### Columns:
- property_id: Unique identifier for each property.
- request_date: Timestamp of user interaction.
- #### Shape: 170,611 rows and 2 columns.
- #### Non-Null Count: Both columns are fully populated.

#### Data Processing Updates:
- #### 1. Photo Count Extraction:

- Extracted the number of photos for each property from the photo_urls column.
- Handled missing and malformed data, setting the count to 0 where necessary.

- #### 2. Total Interactions Calculation:

- Aggregated interaction counts for each property based on the request_date column.

- #### 3. Merged Data:

- Combined the cleaned photo_count data with the total_interactions data.
- Resulting DataFrame has:
- #### 28,888 rows.

- **3 columns**: property_id, photo_count, and total_interactions.
- Some missing values in the total_interactions column where properties lacked interaction data.

#### Analysis Results:

- 1. Percentage of Properties in HSR Layout: It seems that no properties in the dataset have identifiers explicitly matching "HSR" in their property_id. This could be due to inconsistencies in the dataset or a mismatch in assumptions about the data format.

- 2. Number of Unique Locations: Based on property_id, there appears to be only one unique location. This suggests that location information may not be encoded directly in the property_id column as expected.

***Thank You!***